In [1]:
import heapq

class Solution(object):
    def modifiedGraphEdges(self, n, edges, source, destination, target):
        def dijkstra(graph, source):
            dist = [float('inf')] * n
            dist[source] = 0
            heap = [(0, source)]
            visited = set()

            while heap:
                d, node = heapq.heappop(heap)

                if node in visited:
                    continue

                visited.add(node)

                if d > dist[node]:
                    continue

                for neighbor, weight in graph[node]:
                    new_dist = dist[node] + weight
                    if new_dist < dist[neighbor]:
                        dist[neighbor] = new_dist
                        heapq.heappush(heap, (new_dist, neighbor))

            return dist

        # Initial graph setup (ignoring -1 weights for now)
        graph = [[] for _ in range(n)]
        for u, v, w in edges:
            if w != -1:
                graph[u].append((v, w))
                graph[v].append((u, w))

        # Step 1: Compute the initial distance with all -1 edges set to 1
        initial_graph = [g[:] for g in graph]
        for i, (u, v, w) in enumerate(edges):
            if w == -1:
                initial_graph[u].append((v, 1))
                initial_graph[v].append((u, 1))

        initial_dist = dijkstra(initial_graph, source)[destination]

        # If initial distance is greater than target, it's impossible to meet the target
        if initial_dist > target:
            return []

        # If initial distance equals target, no need to modify any -1 edge
        if initial_dist == target:
            return [[u, v, w if w != -1 else 1] for u, v, w in edges]

        # Step 2: If initial_dist < target, check if it's possible to reach exactly target
        # We need to increase some weights to match the target.
        for i, (u, v, w) in enumerate(edges):
            if w == -1:
                # Set this edge to the maximum value 2 * 10^9
                graph[u].append((v, 2 * 10**9))
                graph[v].append((u, 2 * 10**9))

        max_dist = dijkstra(graph, source)[destination]

        # If the maximum possible distance is less than the target, it's impossible
        if max_dist < target:
            return []

        # Binary search on the last modifiable edge to match the target exactly
        for i, (u, v, w) in enumerate(edges):
            if w == -1:
                # We need to binary search for this edge's value
                low, high = 1, 2 * 10**9
                while low < high:
                    mid = (low + high) // 2
                    # Temporarily modify the graph
                    graph[u].remove((v, 2 * 10**9))
                    graph[v].remove((u, 2 * 10**9))
                    graph[u].append((v, mid))
                    graph[v].append((u, mid))

                    mid_dist = dijkstra(graph, source)[destination]

                    if mid_dist < target:
                        low = mid + 1
                    else:
                        high = mid

                    # Restore the graph for the next iteration
                    graph[u].remove((v, mid))
                    graph[v].remove((u, mid))
                    graph[u].append((v, 2 * 10**9))
                    graph[v].append((u, 2 * 10**9))

                # Set the edge to the correct value
                edges[i][2] = low
                graph[u].remove((v, 2 * 10**9))
                graph[v].remove((u, 2 * 10**9))
                graph[u].append((v, low))
                graph[v].append((u, low))

        final_dist = dijkstra(graph, source)[destination]

        if final_dist == target:
            return edges
        else:
            return []